In [1]:
from transitions import Machine
import random

In [2]:
# Setup state machine for zone
class Zone:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.name = f"Zone ({x},{y})"

In [3]:
# Setup state machine for Rider 
class Rider:
    states = ['waiting', 'matched', 'in_ride', 'ride_completed', 'ride_canceled']

    def __init__(self, name, zone):
        self.name = name
        self.zone = zone
        self.estimated_wait_time = 0
        self.machine = Machine(model=self, states=Rider.states, initial='waiting')
        self.machine.add_transition('match_with_driver', 'waiting', 'matched')
        self.machine.add_transition('start_ride', 'matched', 'in_ride')
        self.machine.add_transition('complete_ride', 'in_ride', 'ride_completed')
        self.machine.add_transition('cancel_ride', '*', 'ride_canceled')

    def __str__(self):
        return f"{self.name} (State: {self.state}, Zone: {self.zone.name}, Estimated wait time: {self.estimated_wait_time})"

In [5]:
# Setup state machine for Driver
class Driver:
    states = ['idle', 'en_route', 'waiting_for_rider', 'in_ride', 'ride_completed']

    def __init__(self, name, zone):
        self.name = name
        self.zone = zone
        self.time_to_next_zone = 0
        self.machine = Machine(model=self, states=Driver.states, initial='idle')
        self.machine.add_transition('accept_ride', 'idle', 'en_route')
        self.machine.add_transition('arrive_at_pickup', 'en_route', 'waiting_for_rider')
        self.machine.add_transition('pick_up_rider', 'waiting_for_rider', 'in_ride')
        self.machine.add_transition('complete_ride', 'in_ride', 'ride_completed')
        self.machine.add_transition('reset', '*', 'idle')

    def __str__(self):
        return f"{self.name} (State: {self.state}, Zone: {self.zone.name}, Time to next zone: {self.time_to_next_zone})"

In [6]:
# Setup state machine for RideRequest
class RideRequest:
    states = ['requested', 'accepted', 'in_progress', 'completed', 'canceled']

    def __init__(self, origin_zone, destination_zone):
        self.origin_zone = origin_zone
        self.destination_zone = destination_zone
        self.machine = Machine(model=self, states=RideRequest.states, initial='requested')
        self.machine.add_transition('accept', 'requested', 'accepted')
        self.machine.add_transition('start', 'accepted', 'in_progress')
        self.machine.add_transition('complete', 'in_progress', 'completed')
        self.machine.add_transition('cancel', '*', 'canceled')

In [8]:
# Create the 10x10 grid of zones
zones = {(x, y): Zone(x, y) for x in range(1, 11) for y in range(1, 11)}
# Setup instances of each class
riders = [Rider(f'Rider {i}', random.choice(list(zones.values()))) for i in range(5)]
drivers = [Driver(f'Driver {i}', random.choice(list(zones.values()))) for i in range(3)]
ride_requests = [RideRequest(random.choice(list(zones.values())), random.choice(list(zones.values()))) for i in range(5)]

In [9]:
# Define a function to calculate Manhattan distance
def calculate_travel_time(zone1, zone2):
    return abs(zone1[0] - zone2[0]) + abs(zone1[1] - zone2[1])

# Function to find the nearest available driver
def find_nearest_driver(rider_zone):
    nearest_driver = None
    min_travel_time = float('inf')
    for driver in drivers:
        if driver.state == 'idle':
            travel_time = calculate_travel_time((driver.zone.x, driver.zone.y), (rider_zone.x, rider_zone.y))
            if travel_time < min_travel_time:
                min_travel_time = travel_time
                nearest_driver = driver
    return nearest_driver, min_travel_time

In [10]:
# Run simulation
def simulate_rideshare():
    time_unit = 0
    while any(rider.state != 'ride_completed' for rider in riders):
        print(f"\nTime Unit: {time_unit}")

        for rider in riders:
            if rider.state == 'waiting':
                # Find the nearest driver
                nearest_driver, travel_time = find_nearest_driver(rider.zone)
                if nearest_driver:
                    print(f"{nearest_driver.name} will pick up {rider.name} from {rider.zone.name}. Travel time: {travel_time}")
                    nearest_driver.accept_ride()
                    nearest_driver.time_to_next_zone = travel_time
                    nearest_driver.zone = rider.zone
                    nearest_driver.current_rider = rider
                    nearest_driver.current_request = ride_requests[riders.index(rider)]
                    nearest_driver.arrive_at_pickup()
                    ride_requests[riders.index(rider)].accept()
                    rider.match_with_driver()
                else:
                    rider.estimated_wait_time = travel_time
                    print(f"No driver available for {rider.name}. Estimated wait time: {travel_time} units.")

        for driver in drivers:
            if driver.time_to_next_zone > 0:
                driver.time_to_next_zone -= 1
            elif driver.state == 'waiting_for_rider' and driver.current_rider:
                driver.pick_up_rider()
                driver.current_request.start()
                driver.current_rider.start_ride()
            elif driver.state == 'in_ride' and driver.current_rider:
                driver.complete_ride()
                driver.current_request.complete()
                driver.current_rider.complete_ride()
                driver.current_rider = None
                driver.current_request = None
                driver.reset()

        # Print the states of all riders and drivers
        for rider in riders:
            print(rider)
        for driver in drivers:
            print(driver)

        time_unit += 1

In [11]:
simulate_rideshare()


Time Unit: 0
Driver 0 will pick up Rider 0 from Zone (10,4). Travel time: 3
Driver 2 will pick up Rider 1 from Zone (9,8). Travel time: 5
Driver 1 will pick up Rider 2 from Zone (7,3). Travel time: 2
No driver available for Rider 3. Estimated wait time: inf units.
No driver available for Rider 4. Estimated wait time: inf units.
Rider 0 (State: matched, Zone: Zone (10,4), Estimated wait time: 0)
Rider 1 (State: matched, Zone: Zone (9,8), Estimated wait time: 0)
Rider 2 (State: matched, Zone: Zone (7,3), Estimated wait time: 0)
Rider 3 (State: waiting, Zone: Zone (1,9), Estimated wait time: inf)
Rider 4 (State: waiting, Zone: Zone (8,9), Estimated wait time: inf)
Driver 0 (State: in_ride, Zone: Zone (10,4), Time to next zone: 2)
Driver 1 (State: in_ride, Zone: Zone (7,3), Time to next zone: 1)
Driver 2 (State: in_ride, Zone: Zone (9,8), Time to next zone: 4)

Time Unit: 1
No driver available for Rider 3. Estimated wait time: inf units.
No driver available for Rider 4. Estimated wait tim

MachineError: "Can't trigger event complete from state accepted!"